<a href="https://www.kaggle.com/gurjotkaur25/101903339-gurjotkaur-ds1?scriptVersionId=88960173" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np 
import pandas as pd 
import cudf as cpd
import cupy as cnp
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
dftest = cpd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
dftest.head()
dftrain = cpd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
dftrain.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,<NA>,Reg,Lvl,AllPub,...,0,<NA>,<NA>,<NA>,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,<NA>,Reg,Lvl,AllPub,...,0,<NA>,<NA>,<NA>,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,<NA>,IR1,Lvl,AllPub,...,0,<NA>,<NA>,<NA>,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,<NA>,IR1,Lvl,AllPub,...,0,<NA>,<NA>,<NA>,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,<NA>,IR1,Lvl,AllPub,...,0,<NA>,<NA>,<NA>,0,12,2008,WD,Normal,250000


In [3]:
dftrain.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object

**DATA PRE-PROCESSING**

In [4]:
dftrain.isnull().sum().sort_values(ascending=False)

PoolQC           1453
MiscFeature      1406
Alley            1369
Fence            1179
FireplaceQu       690
                 ... 
MoSold              0
YrSold              0
SaleType            0
SaleCondition       0
SalePrice           0
Length: 81, dtype: uint64

In [5]:
def cleaning_data_none(dftrain: dict, fields: dict):
    for field in fields:
        dftrain[field].fillna('None', inplace=True)

def cleaning_data_int(dftrain: dict, fields: dict):
    for field in fields:
        dftrain[field].fillna(0, inplace=True)

def cleaning_data_median(dftrain: dict, fields: dict):
    for field in fields:
        dftrain[field].fillna(dftrain[field].median(), inplace=True)

In [6]:
fields_clean_none= ['PoolQC','Alley',
                     'FireplaceQu',
                     'MasVnrType',
                     'Electrical',
                     'BsmtFinType2',
                     'BsmtFinType1',
                     'BsmtExposure',
                     'BsmtQual',
                     'BsmtCond',
                     'Fence',
                     'MiscFeature',
                     'GarageCond',
                     'GarageQual',
                     'GarageFinish',
                     'GarageType',
                     'SaleType',
                     'Utilities',
                     'Exterior1st',
                     'Exterior2nd',
                     'KitchenQual',
                     'Functional']

fields_clean_int = ['GarageYrBlt', 'MSZoning', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath']

fields_clean_median = ['LotFrontage',
                        'MasVnrArea',
                        'BsmtUnfSF',
                        'TotalBsmtSF',
                         'GarageCars',
                         'GarageArea']
                       

cleaning_data_none(dftrain, fields_clean_none)
cleaning_data_int(dftrain, fields_clean_int)
cleaning_data_median(dftrain, fields_clean_median)

In [7]:
features=dftrain.columns
features=list(features[1:len(features)-1])

In [8]:
len(features)

79

In [9]:
features

['MSSubClass',
 'MSZoning',
 'LotFrontage',
 'LotArea',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'Enc

In [10]:
dftrain.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


In [11]:
df_types = pd.DataFrame(dftrain.dtypes, columns=["types"])
df_types_object = df_types[df_types["types"] == "object"]
for field_obj in df_types_object.index:
    dftrain[field_obj] = dftrain[field_obj].astype('category').cat.codes
dftrain.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,3,65.0,8450,1,1,3,3,0,...,0,3,4,1,0,2,2008,8,4,208500
1,2,20,3,80.0,9600,1,1,3,3,0,...,0,3,4,1,0,5,2007,8,4,181500
2,3,60,3,68.0,11250,1,1,0,3,0,...,0,3,4,1,0,9,2008,8,4,223500
3,4,70,3,60.0,9550,1,1,0,3,0,...,0,3,4,1,0,2,2006,8,0,140000
4,5,60,3,84.0,14260,1,1,0,3,0,...,0,3,4,1,0,12,2008,8,4,250000


In [12]:
dftrain['SalePrice'] = dftrain['SalePrice'].astype('float64')

In [13]:
Y = dftrain['SalePrice']
Y = cnp.array(Y)
Y.reshape(-1,1)
Y

array([208500., 181500., 223500., ..., 266500., 142125., 147500.])

In [14]:
X = dftrain[features]
X

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,3,65.0,8450,1,1,3,3,0,4,...,0,0,3,4,1,0,2,2008,8,4
1,20,3,80.0,9600,1,1,3,3,0,2,...,0,0,3,4,1,0,5,2007,8,4
2,60,3,68.0,11250,1,1,0,3,0,4,...,0,0,3,4,1,0,9,2008,8,4
3,70,3,60.0,9550,1,1,0,3,0,0,...,0,0,3,4,1,0,2,2006,8,0
4,60,3,84.0,14260,1,1,0,3,0,2,...,0,0,3,4,1,0,12,2008,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,3,62.0,7917,1,1,3,3,0,4,...,0,0,3,4,1,0,8,2007,8,4
1456,20,3,85.0,13175,1,1,3,3,0,4,...,0,0,3,2,1,0,2,2010,8,4
1457,70,3,66.0,9042,1,1,3,3,0,4,...,0,0,3,0,3,2500,5,2010,8,4
1458,20,3,68.0,9717,1,1,3,3,0,4,...,0,0,3,4,1,0,4,2010,8,4


In [15]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_scaled = scaler.fit_transform(X.as_matrix())

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_scaled, Y, test_size = 0.3, random_state = 42)

In [17]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(1022, 79)
(438, 79)
(1022,)
(438,)


In [18]:
import cuml
from cuml import LinearRegression

In [19]:
X_test

array([[-0.87256276, -0.04553194,  0.00618985, ..., -1.36765473,
         0.31386709,  0.2085023 ],
       [ 0.07337496, -0.04553194,  1.27775352, ...,  1.64520971,
         0.31386709,  0.2085023 ],
       [-0.63607833,  1.53724502, -0.62959198, ...,  1.64520971,
         0.31386709,  0.2085023 ],
       ...,
       [ 0.07337496, -3.21108585, -0.22087509, ...,  0.13877749,
         0.31386709,  0.2085023 ],
       [-0.16310947, -0.04553194, -0.13004911, ...,  0.8919936 ,
        -2.26416922,  0.2085023 ],
       [ 0.07337496, -0.04553194,  1.55023145, ...,  0.8919936 ,
        -0.97515106,  1.11719888]])

**Using Linear Regression**

In [20]:
lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'eig')
reg = lr.fit(X_train, Y_train)
print("Coefficients:")
print(reg.coef_)
print("Intercept:")
print(reg.intercept_)
print("Predictions: ")
pred = lr.predict(X_test)
print(pred)
print("MSE: ")
print(cuml.metrics.regression.mean_squared_error(Y_test,pred))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(Y_test,pred))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(Y_test,pred))

Coefficients:
[ -5855.12187355  -1059.36938469  -4909.11850242   3537.81585952
   1391.30605537   1581.99176137  -1415.49755924   2436.76969184
  -1514.83544762    227.35544699   3102.62487358   2659.97555517
   -623.30620327  -4494.64035488  -2978.70975533  -3483.28587144
  15485.8025317    5335.5780853    6289.03901559    149.60285215
   2174.56435805   3106.04054534  -4881.31997679   2660.8936167
   2114.60436671   4473.14618194  -6364.52517353    678.3082903
    518.45990961  -7300.42553887   2432.12641817  -4504.91438644
  -1569.64831591    815.6852843     989.15890823    691.15498516
  -1848.67285861   -759.82284299   -947.53360687   -635.26834551
    183.50470116   -384.30324356   7987.28429378   7664.1446225
  -1375.06763188  12115.66294023   4364.13082931   -339.38973296
   1237.79648599   -403.25635722  -1982.73540038  -3319.92159267
  -7718.55663206   7945.82490154   4137.07069513   4052.67726763
  -1210.58371728   -945.99943403  -4061.29630986     87.17281171
  11706.789649

**Linear regression algorithms**

In [21]:
algorithm = ['svd', 'eig', 'qr', 'svd-qr', 'svd-jacobi']
print('\033[1m',"Generating comparison of MSE, MAE, R2-score for all the five Linear Regression algorithms ",'\033[0m')
for i in algorithm:
    print("Algorithm:")
    print(i)
    lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = i)
    reg = lr.fit(X_train,Y_train)
    preds = lr.predict(X_test)
    print("MSE:")
    print(cuml.metrics.regression.mean_squared_error(Y_test,preds))
    print("R2 Score:")
    print(cuml.metrics.regression.r2_score(Y_test,preds))
    print("MAE:")
    print(cuml.metrics.regression.mean_absolute_error(Y_test,preds))

 Generating comparison of MSE, MAE, R2-score for all the five Linear Regression algorithms  
Algorithm:
svd
MSE:
1084070967.9947193
R2 Score:
0.844646546014768
MAE:
21873.24614382635
Algorithm:
eig
MSE:
1084070967.994126
R2 Score:
0.844646546014853
MAE:
21873.246143826007
Algorithm:
qr
MSE:
1071237580.3186837
R2 Score:
0.846485642495224
MAE:
21725.578795391633
Algorithm:
svd-qr
MSE:
2.9743476065761283e+33
R2 Score:
-4.262407053378082e+23
MAE:
4.345221127079089e+16
Algorithm:
svd-jacobi
MSE:
2.974347606576188e+33
R2 Score:
-4.262407053378169e+23
MAE:
4.345221127079123e+16
